# Think Bayes

Copyright 2018 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [38]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import numpy as np
import pandas as pd

from scipy.stats import poisson

# import classes from thinkbayes2
from thinkbayes2 import Pmf, Cdf, Suite, Joint, EvalPoissonPmf

import thinkbayes2
import thinkplot
import itertools

import pymc3 as pm
import theano.tensor as T

### Fake data

In [10]:
n = 60
t1 = 30
t2 = n-t1
lam1 = 4
lam2 = 2

2

In [11]:
before = poisson(lam1).rvs(t1)

array([3, 4, 4, 1, 5, 8, 1, 5, 3, 2, 4, 4, 6, 1, 4, 2, 4, 8, 2, 4, 3, 2,
       3, 2, 4, 5, 2, 3, 5, 3])

In [12]:
after = poisson(lam2).rvs(t2)

array([3, 1, 1, 2, 5, 1, 2, 3, 4, 1, 2, 1, 2, 5, 2, 4, 1, 3, 2, 3, 1, 3,
       1, 2, 2, 2, 2, 2, 4, 2])

In [13]:
data = np.concatenate([before, after])

array([3, 4, 4, 1, 5, 8, 1, 5, 3, 2, 4, 4, 6, 1, 4, 2, 4, 8, 2, 4, 3, 2,
       3, 2, 4, 5, 2, 3, 5, 3, 3, 1, 1, 2, 5, 1, 2, 3, 4, 1, 2, 1, 2, 5,
       2, 4, 1, 3, 2, 3, 1, 3, 1, 2, 2, 2, 2, 2, 4, 2])

### Grid algorithm

In [35]:
class Change(Suite, Joint):
    
    def Likelihood(self, data, hypo):
        """
        
        data: array of counts
        hypo: t, lam1, lam2
        """
        t, lam1, lam2 = hypo
        likes = []
        for i in range(len(data)):
            if i < t:
                likes.append(EvalPoissonPmf(data[i], lam1))
            else:
                likes.append(EvalPoissonPmf(data[i], lam2))
        like = itertools.product(likes)
        return like

In [36]:
cp = Change()
lams = np.linspace(0, 10, 11)
ts = np.linspace(0, 60, 61)
for lam1 in lams:
    for lam2 in lams:
        for t in ts:
            cp.Set((lam1, lam2, t), 1)
cp.Normalize();

In [39]:
cp.Update(data)
lam1_dist = cp.Marginal(0)
lam2_dist = cp.Marginal(1)
tau_dist = cp.Marginal(2)
thinkplot.plot(lam1_dist)
thinkplot.plot(lam2_dist, color='grey')
thinkplot.figure()
thinkplot.plot(tau_dist)

TypeError: unsupported operand type(s) for *: 'float' and 'itertools.product'

### MCMC

To implement this model in PyMC, see Chapter 1 of [Bayesian Methods for Hackers](http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Ch1_Introduction_PyMC2.ipynb)
and this example from [Computational Statistics in Python](http://people.duke.edu/~ccc14/sta-663-2016/16C_PyMC3.html#Changepoint-detection)

In [31]:
niter = 10000
t = range(len(data))
with pm.Model() as change_point:
    cp = pm.DiscreteUniform('change_point', lower=0, upper=len(data), testval=len(data)//2)
    mu0 = pm.Exponential('mu0', 1/data.mean())
    mu1 = pm.Exponential('mu1', 1/data.mean())
    mu = T.switch(t < cp, mu0, mu1)
    Y_obs = pm.Poisson('Y_obs', mu=mu, observed=data)
    trace = pm.sample(niter, tune=2000)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [change_point]
>NUTS: [mu1, mu0]
Sampling 4 chains: 100%|██████████| 48000/48000 [00:12<00:00, 3962.47draws/s]
The acceptance probability does not match the target. It is 0.8791647954690315, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8826940095905892, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 10% for some parameters.


### Real data

Some real data, based on [this analysis from the Baltimore Sun](http://www.baltimoresun.com/news/maryland/crime/bs-md-ci-violence-stats-20181018-story.html)

In [27]:
# !wget https://raw.githubusercontent.com/baltimore-sun-data/2018-shootings-analysis/master/BPD_Part_1_Victim_Based_Crime_Data.csv

--2018-10-26 15:04:21--  https://raw.githubusercontent.com/baltimore-sun-data/2018-shootings-analysis/master/BPD_Part_1_Victim_Based_Crime_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.128.133, 151.101.192.133, 151.101.0.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45658084 (44M) [text/plain]
Saving to: ‘BPD_Part_1_Victim_Based_Crime_Data.csv’

BPD_Part_1_Victim_B 100%[===================>]  43.54M  1.37MB/s    in 49s     

2018-10-26 15:05:10 (916 KB/s) - ‘BPD_Part_1_Victim_Based_Crime_Data.csv’ saved [45658084/45658084]



In [28]:
df = pd.read_csv('BPD_Part_1_Victim_Based_Crime_Data.csv', parse_dates=[0])
df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,crimeCaseNumber,Total Incidents
0,2018-07-07,23:53:00,1F,1600 PENTWOOD RD,HOMICIDE,NaN,FIREARM,413.0,NORTHEASTERN,Stonewood-Pentwood-Winsto,-76.58727,39.34782,"(39.34782, -76.58727)",Alley,NaN,1.0
1,2018-07-07,23:50:00,4E,ST & DIVISION ST,COMMON ASSAULT,O,NaN,131.0,CENTRAL,Druid Heights,-76.63936,39.30903,"(39.30903, -76.63936)",STREET,NaN,1.0
2,2018-07-07,23:18:00,4C,2500 PERRING MANOR RD,AGG. ASSAULT,I,OTHER,423.0,NORTHEASTERN,Hamilton Hills,-76.56094,39.37189,"(39.37189, -76.56094)",ROW/TOWNHO,NaN,1.0
3,2018-07-07,22:41:00,9S,3700 S HANOVER ST,SHOOTING,NaN,FIREARM,913.0,SOUTHERN,Brooklyn,-76.61033,39.23703,"(39.23703, -76.61033)",Common Bus,NaN,1.0
4,2018-07-07,22:55:00,4E,LOMBARD ST & LIGHT ST,COMMON ASSAULT,I,NaN,111.0,CENTRAL,Inner Harbor,-76.61362,39.28775,"(39.28775, -76.61362)",CONVENIENC,NaN,1.0


In [ ]:
df.shape

In [ ]:
shootings = df[df.Description.isin(['HOMICIDE', 'SHOOTING']) & (df.Weapon == 'FIREARM')]
shootings.shape

In [ ]:
grouped = shootings.groupby('CrimeDate')

In [ ]:
counts = grouped['Total Incidents'].sum()
counts.head()

In [ ]:
index = pd.date_range(counts.index[0], counts.index[-1])

In [ ]:
counts = counts.reindex(index, fill_value=0)
counts.head()

In [ ]:
counts.plot()
thinkplot.decorate(xlabel='Date',
                   ylabel='Number of shootings')